In [2]:
# ensure that the imported .py file will get auto imported and updated whenever there is a change
%load_ext autoreload
%autoreload 2

In [3]:
!pip install PyMuPDF

In [4]:

import fitz
import os

In [5]:

project_path = os.getcwd()
project_data_path = project_path + "/data"
project_processed_data_path = project_path + "/processed"
if not os.path.exists(project_data_path):
    os.makedirs(project_data_path)
if not os.path.exists(project_processed_data_path):
    os.makedirs(project_processed_data_path)

In [6]:
import nltk
from nltk.tokenize import word_tokenize #Used to extract words from documents
from nltk.stem import WordNetLemmatizer #Used to lemmatize words
from nltk.corpus import stopwords
import re
import pandas as pd
pd.options.display.max_colwidth = 1000
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')

import processing_pdf
pd.options.display.max_colwidth = 2000

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Claud\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Claud\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Claud\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# pdf_file =  "An Empirical Survey on Long Document Summarization.pdf"
# pdf_file = "1901.00009v1.pdf"
# pdf_file = "1901.00936v3.pdf"
pdf_file = "an image is worth 16 by 16 words.pdf"
doc, total_text, total_pages = processing_pdf.open_file(project_data_path + "/" + pdf_file)
table_of_content = doc.get_toc()
print("Auto generated table of content:")
display(table_of_content)

# some papers have not table of content
if len(table_of_content) == 0:
    print("The paper has not table of content. Need to use regular expression to map table of content.")
    res = processing_pdf.auto_find_toc(doc)
    print("Not satified the generated result or want to adjust? Build your own table of content by using this template:")
    display(res)



Auto generated table of content:


[[1, '1 Introduction', 1],
 [1, '2 Related Work', 2],
 [1, '3 Method', 3],
 [2, '3.1 Vision Transformer (ViT)', 3],
 [2, '3.2 Fine-tuning and Higher Resolution', 4],
 [1, '4 Experiments', 4],
 [2, '4.1 Setup', 4],
 [2, '4.2 Comparison to State of the Art', 5],
 [2, '4.3 Pre-training Data Requirements', 6],
 [2, '4.4 Scaling Study', 8],
 [2, '4.5 Inspecting Vision Transformer', 8],
 [2, '4.6 Self-supervision', 8],
 [1, '5 Conclusion', 9],
 [1, 'A Multihead Self-attention', 13],
 [1, 'B Experiment details', 13],
 [2, 'B.1 Training', 13],
 [3, 'B.1.1 Fine-tuning', 13],
 [3, 'B.1.2 Self-supervision', 14],
 [1, 'C Additional Results', 14],
 [1, 'D Additional Analyses', 15],
 [2, 'D.1 SGD vs. Adam for ResNets', 15],
 [2, 'D.2 Transformer shape', 16],
 [2, 'D.3 Head Type and class token', 16],
 [2, 'D.4 Positional Embedding', 17],
 [2, 'D.5 Empirical Computational Costs', 18],
 [2, 'D.6 Axial Attention', 19],
 [2, 'D.7 Attention Distance', 20],
 [2, 'D.8 Attention Maps', 20],
 [2, 'D.9 Object

In [9]:
#uncomment this list to customize table-of-content
# table_of_content = [[1, 'I. INTRODUCTION', 1],
#  [1, 'II. SFC BASED ON IPV6 SEGMENT ROUTING', 2],
#  [1, 'III. DESIGN OF THE SRV6 PROXY', 4],
#  [2, 'A. General Concepts and State-of-the-art', 4],
#  [2, 'B. SRNKv1', 5],
#  [2, 'C. SRNKv2', 7],
#  [2, 'D. Implementation of other SR proxy types', 8],
#  [1, 'IV. TESTING ENVIRONMENT', 8],
#  [1, 'V. PERFORMANCE ANALYSIS', 9],
#  [1, 'VII. CONCLUSIONS', 11]]

# separate content into sections
processing_pdf.clear_processed_folder(project_processed_data_path)
ds, json_dict = processing_pdf.separate_content(total_text, table_of_content)
processing_pdf.save_dataframe(ds, json_dict, project_processed_data_path,  pdf_file.rsplit(".", 1)[0])
# extract images
processing_pdf.find_images(doc, table_of_content, total_pages, project_processed_data_path)

display(ds)


starting looking for all the sections according to the provided section title info...
save the dataframe to d:\Start_Dec_19_2023\application\NUS Course\Semester_1\Neural Network and Deep Learning\PDF_Conversion/processed/an image is worth 16 by 16 words.csv
save the dataframe to d:\Start_Dec_19_2023\application\NUS Course\Semester_1\Neural Network and Deep Learning\PDF_Conversion/processed/an image is worth 16 by 16 words.json


,level_1,level_1_content,level_2,level_2_content,level_3,level_3_content
0,NA,"published as a conference paper at iclr an image is worth x words: transformers for image recognition at scale alexey dosovitskiy,, lucas beyer, alexander kolesnikov, dirk weissenborn, xiaohua zhai, thomas unterthiner, mostafa dehghani, matthias minderer, georg heigold, sylvain gelly, jakob uszkoreit, neil houlsby, equal technical contribution, equal advising google research, brain team {adosovitskiy, neilhoulsby}@google.com abstract while the transformer architecture has become the de facto standard for natural language processing tasks, its applications to computer vision remain limited. in vision, attention is either applied in conjunction with convolutional networks, or used to replace certain components of convolutional networks while keeping their overall structure in place. we show that this reliance on cnns is not necessary and a pure transformer applied directly to sequences of image patches can perform very well on image classication tasks. when pre trained on large amounts of data and transferred to multiple mid sized or small image recognition benchmarks (imagenet, cifar-, vtab, etc, vision transformer (vit) attains excellent results compared to state of the art convolutional networks while requiring sub- stantially fewer computational resources to train. introduction",NaN,NaN,NaN,NaN
1,1 Introduction,"self attention based architectures, in particular transformers (vaswani et al., ), have become the model of choice in natural language processing (nlp). the dominant approach is to pre train on a large text corpus and then ne tune on a smaller task specic dataset (devlin et al., ). thanks to transformers computational efciency and scalability, it has become possible to train models of unprecedented size, with over b parameters (brown et al., ; lepikhin et al., ). with the models and datasets growing, there is still no sign of saturating performance. in computer vision, however, convolutional architectures remain dominant (lecun et al., ; krizhevsky et al., ; he et al., ). inspired by nlp successes, multiple works try combining cnn like architectures with self attention (wang et al., ; carion et al., ), some replacing the convolutions entirely (ramachandran et al., ; wang et al., a). the latter models, while theoretically efcient, have not yet been scaled effectively on modern hardware accelerators due to the use of specialized attention patterns. therefore, in large scale image recognition, classic resnet- like architectures are still state of the art (mahajan et al., ; xie et al., ; kolesnikov et al., ). inspired by the transformer scaling successes in nlp, we experiment with applying a standard transformer directly to images, with the fewest possible modications. to do so, we split an image into patches and provide the sequence of linear embeddings of these patches as an input to a trans- former. image patches are treated the same way as tokens (words) in an nlp application. we train the model on image classication in supervised fashion. when trained on mid sized datasets such as imagenet without strong regularization, these mod- els yield modest accuracies of a few percentage points below resnets of comparable size. this seemingly discouraging outcome may be expected: transformers lack some of the inductive biases fine tuning code and pre trained models are a...",NaN,NaN,NaN,NaN
2,2 Related Work,"transformers were proposed by vaswani et al. () for machine translation, and have since be- come the state of the art method in many nlp tasks. large transformer based models are often pre trained on large corpora and then ne tuned for the task at hand: bert (devlin et al., ) uses a denoising self supervised pre training task, while the gpt line of work uses language mod- eling as its pre training task (radford et al., ; ; brown et al., ). naive application of self attention to images would require that each pixel attends to every other pixel. 

In [ ]:
import json

# open json file
with open(project_processed_data_path + "/1901.00936v3.json") as f:
    data = json.load(f)
    for item in data:
        display(item)

In [26]:
subsec_pattern_2 = re.compile(r'(\r|\n)([A-E]\.(\n|\r|\s)+([a-zA-Z-]+\s)+)')
text = '''
its performance improvements. 
A. General Concepts and State-of-the-art
1) Network Programming Model: The SRv6 network programming
model [11] extends the IPv6 Segment Routing
concept from the simple steering
'''
match = subsec_pattern_2.search(text)
print(match)

<re.Match object; span=(31, 73), match='\nA. General Concepts and State-of-the-art\n'>
